# Advent of Code 2025 Solutions

# [Day 1: Secret Entrance](https://adventofcode.com/2025/day/1)

## Part 1

In [1]:
with open('input1.txt', 'r') as file:
    input_text = file.read()

In [2]:
def get_rotations(input_text):
    lines = input_text.strip().splitlines()
    return lines

We just need to keep track of where our `dial` is pointing. Since the dial is a circle, we can look at the remainder $\mod{100}$ everytime we rotate our dial by a certain amount. Every time our dial points at `0` after a rotation, we add one to our counter.

In [3]:
rotations = get_rotations(input_text)
dial = 50
zero_counter = 0
for rotation in rotations:
    direction = rotation[0]
    distance = int(rotation[1:])
    shift = distance if direction == 'R' else - distance
    dial = (dial + shift) % 100
    if dial == 0:
        zero_counter += 1

In [4]:
zero_counter

1177

## Part 2

For this part, we add or subtract the appropriate distance based on the direction of our rotation and keep track of this value *without* modding by 100. Let `start` be the state of our dial before a rotation and `end` be the state of our dial after a rotation. Then the number of clicks that occur during this rotation corresponds to the number of multiples of 100 between our `start` and `end`. We can count this in most cases with $$\left \vert \left \lfloor \frac{\textrm{start}}{100} \right\rfloor - \left\lfloor \frac{\textrm{end}}{100} \right\rfloor \right \vert.$$

The caveat is that if we are rotating left, and we start on a multiple of 100, this method will count an additional click that we do not want to count. For example, if `start = 100` and our rotation is `L5` to get `end = 95`, this corresponds to our `dial` starting at `0` and ending at `95`. We do not want count a click in this rotation, but $$\left \lfloor \frac{100}{100} \right\rfloor - \left\lfloor \frac{95}{100} \right\rfloor = 1 - 0 = 1,$$
so our initial counting method would count 1 click during this rotation.

Similarly, if we are rotating left, and we end on a multiple of 100, we want to count this end state as a click even though `floor(start/100) - floor(end/100)` will not detect this. For example, consider `start = 340` with rotation `L40` to give us `end = 200`. This corresponds to our `dial` starting at `40`, wrapping all the way around once, and then rotating further to end at `0`. This means we should count `2` clicks in this rotation, but $$\left \lfloor \frac{340}{100} \right\rfloor - \left\lfloor \frac{200}{100} \right\rfloor = 3 - 2 = 1.$$ To fix both of these edge cases, we add in two if statements to adjust the number of clicks in a left-rotation accordingly.

In [5]:
dial = 50
click_counter = 0
for rotation in rotations:
    direction = rotation[0]
    distance = int(rotation[1:])
    
    if direction == 'R':
        start = dial
        dial += distance
        end = dial
    else:
        start = dial
        if start % 100 == 0:
            start -= 1
        dial -= distance
        end = dial
        if end % 100 == 0:
            end -= 1
    click_counter += abs(start//100 - end//100)

In [6]:
click_counter

6768

# [Day 2: Gift Shop](https://adventofcode.com/2025/day/2)

## Part 1

In [1]:
with open('input2.txt', 'r') as file:
    input_text = file.read()

In [33]:
input_text = "11-22,95-115,998-1012,1188511880-1188511890,222220-222224,1698522-1698528,446443-446449,38593856-38593862,565653-565659,824824821-824824827,2121212118-2121212124"

In [2]:
def get_id_ranges(input_text):
    ranges = input_text.strip().split(',')
    ranges = [[int(bound) for bound in bounds.split('-')] for bounds in ranges]
    return ranges

To check if an ID is valid, we split the ID into two equal halves and check if they are equal. Note that this means any odd length ID will automatically be valid.

In [3]:
def is_valid_id(ID):
    id_str = str(ID)
    n = len(id_str)
    if n % 2 != 0:
        return True
    first = id_str[:n//2]
    second = id_str[n//2:]
    return first != second

In [4]:
id_ranges = get_id_ranges(input_text)
total = 0
for id_range in id_ranges:
    lower, upper = id_range
    for i in range(lower, upper + 1):
        if not is_valid_id(i):
            total += i

In [5]:
total

34826702005

## Part 2

Let the length of our ID be $n$. For every $1 \leq k \leq \frac{n}{2}$ divding $n$, we can divide the ID into $m = \frac{n}{k}$ equal substrings of length $k$. This ID is invalid if the $m$ substrings of length $k$ are equal for any $k$ dividing $n$. Otherwise, the ID is valid.

In [6]:
def k_repetitions(id_str, n, k):
    m = n//k
    first = id_str[:k]
    for i in range(1, m):
        if first != id_str[i*k:(i+1)*k]:
            return False
    return True

def is_valid_id_2(ID):
    id_str = str(ID)
    n = len(id_str)
    for k in range(1, n//2+1):
        if n % k == 0 and k_repetitions(id_str,n,k):
            return False
    return True

In [7]:
id_ranges = get_id_ranges(input_text)
total = 0
for id_range in id_ranges:
    lower, upper = id_range
    for i in range(lower, upper + 1):
        if not is_valid_id_2(i):
            total += i

In [8]:
total

43287141963

# [Day 3: Lobby](https://adventofcode.com/2025/day/3)

## Part 1

In [1]:
with open('input3.txt', 'r') as file:
    input_text = file.read()

In [2]:
def get_banks(input_text):
    banks = input_text.strip().splitlines()
    banks = [[int(battery) for battery in bank]for bank in banks]
    return banks

To figure out the first battery to turn on, we want to choose the largest joltage battery that is not last in the bank. Moreover, if there are multiple batteries in the bank that attain the maximum joltage, we want to choose the leftmost one, so that way we can select another maximum joltage battery as our second battery.

In [3]:
def get_max_joltage(bank):
    first = 0
    first_idx = 0
    for i, battery in enumerate(bank[:-1]):
        if battery > first:
            first = battery
            first_idx = i
            if battery == 9:
                break
    
    second = 0
    second_idx = first_idx + 1
    for i, battery in enumerate(bank[second_idx:]):
        if battery > second:
            second = battery
            second_idx = i
            if battery == 9:
                break
    return (first * 10) + second

In [4]:
banks = get_banks(input_text)
total_joltage = 0
for bank in banks:
    total_joltage += get_max_joltage(bank)

In [5]:
total_joltage

17155

## Part 2

We apply the same algorithm here. To choose our first battery, we want to choose the battery with the largest joltage that is not in the rightmost 11 positions of the bank. Again, if there are multiple maximum attaining batteries, we choose the leftmost one. Once the $k^\textrm{th}$ battery has been selected, we  want to choose the largest joltage battery that is not in the rightmost $12 - (k + 1)$ positions while still being to the right of the $k^\textrm{th}$ battery selected. Again, if multiple batteries satisfy all these conditions, we select the $(k+1)^\textrm{th}$ battery to be the leftmost battery satisfying these conditions. The importance of choosing the leftmost, maximal, valid battery in each step allows for the largest possible selection of valid batteries for the next choice of battery.

If we naively follow this algorithm like we did in Part 1, we will have to iterate through each bank 12 times, giving us a $n^{12}$ runtime where $n$ is the number of batteries in each bank. To execute our algorithm with one pass through per bank, we keep a dictionary with keys given by joltage levels `1` to `9` and values given by the lists of indices of batteries in the bank that have that joltage. Then to select the $k^\textrm{th}$ battery, we can iterate starting from `9` down until we have a valid battery i.e. one that is to the right of the $k^\textrm{th}$ and to the left of at least $12 - (k+1)$ batteries.

First, let's implement `find_minimum_valid_index`, which for a given list of `indices` in our bank with a fixed voltage, finds the smallest index that is to the left of the `current_index` of the previously selected battery. If there is no valid index in `indices` we return `-1`. We can implement this in $\log n$ time using binary search since `indices` is already sorted from lowest to highest.

In [6]:
def find_minimum_valid_index(indices, current_index):
    left = 0
    right = len(indices) - 1
    while left <= right:
        middle = (left + right) // 2
        if indices[middle] > current_index and (middle == left or indices[middle-1] <= current_index):
            return middle
        elif indices[middle] > current_index and indices[middle-1] > current_index :
            right = middle - 1
        else:
            left = middle + 1
    return -1

Now we can run the rest of our algorithm. Since we're iterating by descending joltage value, the first valid battery that we find will be of maximum joltage. Note that `find_minimum_valid_index` only checks to see if there exists a battery with the specified voltage to the left of our most recently chosen battery. We must verify that this battery is to the left of at least $12 - (k+1)$ batteries separately.

In [7]:
def get_max_joltage_2(bank):
    joltage_indices = {n: [] for n in range(1,10)}
    for i, battery in enumerate(bank):
        joltage_indices[battery].append(i)

    joltage = ""
    bank_length = len(bank)
    current_index = -1
    while len(joltage) < 12:
        for n in range(9,0,-1):
            if not joltage_indices:
                continue
            min_index = find_minimum_valid_index(joltage_indices[n], current_index)
            if min_index == -1:
                continue
            if bank_length - joltage_indices[n][min_index] >= 12 - len(joltage):
                joltage += str(n)
                current_index = joltage_indices[n][min_index]
                break
    return int(joltage)

In [8]:
total_joltage = 0
for bank in banks:
    total_joltage += get_max_joltage_2(bank)
total_joltage

169685670469164

# [Day 4: Printing Department](https://adventofcode.com/2025/day/4)

## Part 1

In [1]:
with open('input4.txt', 'r') as file:
    input_text = file.read()

First, let's parse our grid and pad it so we don't have to worry about our indices going out of bounds later.

In [2]:
def read_grid(input_text):
    rows = input_text.strip().splitlines()
    grid = [['.' for _ in range(len(rows[0])+2)]]
    for row in rows:
        grid_row = ['.']
        grid_row.extend([char for char in row])
        grid_row.append('.')
        grid.append(grid_row)
    grid.append(['.' for _ in range(len(rows[0])+2)])
    return grid

Now let's write a function to count the number of rolls adjacent to `grid[i][j]`.

In [3]:
def count_neighbors(grid, i, j):
    rolls = 0
    for k in range(i-1, i+2):
        for l in range(j-1, j+2):
            if grid[k][l] == "@" and (k,l) != (i,j):
                rolls += 1
    return rolls

Now we just count the total number of rolls with less than 4 adjacent rolls.

In [4]:
grid = read_grid(input_text)
accessible_rolls = 0
for i, row in enumerate(grid):
    if i in [0, len(grid)]:
        continue
    for j, char in enumerate(row):
        if j in [0, len(row)]:
            continue
        if char == "@" and count_neighbors(grid, i, j) < 4:
            accessible_rolls += 1
accessible_rolls

1478

## Part 2

We can repeat what we did in Part 1, but now remove any roll with less than 4 adjacent rolls. We repeat this until we cannot remove any more rolls.

In [5]:
def remove_rolls(grid):
    removable_rolls = 0
    for i, row in enumerate(grid):
        if i in [0, len(grid)]:
            continue
        for j, char in enumerate(row):
            if j in [0, len(row)]:
                continue
            if char == "@" and count_neighbors(grid, i, j) < 4:
                grid[i][j] = "."
                removable_rolls += 1
    return removable_rolls

In [6]:
removable_rolls = remove_rolls(grid)
total = removable_rolls
while removable_rolls > 0:
    removable_rolls = remove_rolls(grid)
    total += removable_rolls
total

9120